In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llamabot import QueryBot
import git
from IPython.display import display, Markdown

# Eric Ma Q&A

This shows how to build a blog Q&A bot using the text contents of Eric Ma's blog.

## Setup: Download blog data

In [ ]:
import tempfile
from pathlib import Path

# Create a temporary directory
temp_dir = tempfile.TemporaryDirectory(dir="/tmp")

# Clone the repository into the temporary directory
repo = git.Repo.clone_from("https://github.com/ericmjl/website", temp_dir.name)

# Set the root directory to the cloned repository
root_dir = Path(temp_dir.name) / "content" / "blog"

In [ ]:
from pathlib import Path


# Find all *.lr and *.md files recursively in the root directory
def find_files_with_suffixes(directory, suffixes):
    files = []
    for suffix in suffixes:
        files.extend(list(Path(directory).rglob(f"*.{suffix}")))
    return files

In [ ]:
lektor_files

In [ ]:
from pathlib import Path


def file_tree(directory):
    tree = ""
    indent = ""
    for root, dirs, files in Path(directory).rglob("*"):
        tree += f"{indent}{Path(root).name}/\n"
        indent += "  "
        for file in files:
            tree += f"{indent}{file.name}\n"
    return tree


# Example usage
directory = "/path/to/directory"
file_tree = get_file_tree(directory)
print(file_tree)

In [ ]:
from chromadb import QueryResult
from slugify import slugify
import chromadb

client = chromadb.PersistentClient(path=str(Path.home() / ".llamabot" / "chroma.db"))
collection = client.create_collection(
    slugify("https://github.com/ericmjl/website"), get_or_create=True
)

results = collection.get()

In [ ]:
results = collection.get()

In [ ]:
type(results['documents'][0])

In [ ]:
bot = QueryBot(
    system_prompt="You are a digital twin of Eric Ma, the author of the blog posts that you will query.",
    collection_name="eric-ma-blog-posts",
    document_paths=lektor_files,
)

In [ ]:
bot("What is your view on building a digital portfolio?")

In [ ]:
bot("What were your experiences with the SciPy conference?")

In [ ]:
bot("What tutorials did you attend at the SciPy conference in 2023?")

## LlamaBot Code Query

In [ ]:
from numpy import source
from llamabot.file_finder import recursive_find
from pyprojroot import here

source_python_files = recursive_find(root_dir=here() / "llamabot", file_extension=".py")

codebot = QueryBot(
    "You are an expert in code Q&A.",
    collection_name="llamabot",
    document_paths=source_python_files,
    model_name="gpt-4-1106-preview",
)

In [ ]:
codebot("How do I find all the files in a directory?")

In [ ]:
codebot("Which Bot do I use to chat with my documents?")

In [ ]:
codebot("Explain to me the architecture of SimpleBot.")

In [ ]:
codebot("What are the CLI functions available in LlamaBot?")

In [ ]:
from llamabot.bot.qabot import DocQABot

codebot = DocQABot(
    collection_name="llamabot",
)
codebot.add_documents(document_paths=source_python_files)

In [ ]:
codebot(
    "Does LlamaBot provide a function to find all files recursively in a directory?"
)